# Apollo Workspace Setup

This notebook assumes you're working in a environment where `/workspace` is your root directory. Before proceeding:

1. Ensure your Apollo workspace is properly mounted
2. All paths in this notebook are relative to `/workspace`
3. Required files:
   - `model_config.json`
   - `model.ckpt` (vanilla model)
   - `training-dataset.json`
   - Dataset directory with audio files
   - Training metadata CSV

> 💡 **Note**: All commands assume `/workspace` as the current working directory

# Install stable-audio-tools

In [ ]:
%cd /workspace/stable-audio-tools
!pip install .

# Run Inference (Vanilla Model)

This step runs inference using the base model configuration. It will:
1. Load the vanilla model with basic configuration
2. Start a Gradio interface
3. Create a shareable link for testing

> 💡 **Note**: This is useful for testing the base model before training

In [ ]:
%cd /workspace/stable-audio-tools

command = (
    "python run_gradio.py"
    " --model-config ./model_config.json"
    " --ckpt-path ./checkpoints/model-001.ckpt"
    " --share"
)
!{command}

# Training Setup and Configuration

This section covers model training configuration and execution. Key components:

### Weights & Biases Integration (Optional)
- Install and configure W&B for experiment tracking
- Use `wandb login` with your API key

### Training Configuration
- **Dataset**: Defined in `training-dataset.json`
- **Model**: Base configuration in `model_config.json`
- **GPU Usage**: 
  - Default: Single GPU (A100)
  - Multi-GPU scaling available for large datasets (100GB+)
  
### Important Parameters
- **Checkpoints**: 
  - Each checkpoint ≈ 16GB
  - Saved every N steps
  - Storage strategy needed for long runs
- **Batch Size**: Configurable based on GPU memory
- **Training Duration**: 
  - Currently runs indefinitely
  - Manual stopping required
  - Future: Epoch-based training planned

> ⚠️ **Note**: Monitor disk space as checkpoints accumulate

In [ ]:
%cd /workspace/stable-audio-tools

command = (
    "python3 train.py"
    " --dataset-config ./training-dataset.json"
    " --model-config ./model_config.json"
    " --name my_model_name"
    " --save-dir ./checkpoints/"
    " --checkpoint-every 100"
    " --batch-size 32"
    " --num-gpus 1"
    " --precision 16-mixed"
    " --seed 42"
    " --pretrained-ckpt-path ./checkpoints/model-001.ckpt"
)

!{command}

# Unwrap the Model

After training, the model needs to be unwrapped for inference. Important considerations:

### Process Requirements
- Heavy memory usage (≈50GB tested on A6000)
- GPU memory management challenges:
  - PyTorch Lightning memory retention issues
  - May need manual cache clearing/GC if executed directly after training
  
> ⚠️ **Resource Note**: Consider dedicated workers for unwrapping in grid search scenarios

In [ ]:
%cd /workspace/stable-audio-tools

command = (
    "python unwrap_model.py"
    " --model-config ./model_config.json"
    " --ckpt-path ./checkpoints/<my_model_name>/<run_id>/checkpoints/<epoch>.ckpt"
    " --name <my_model_checkpoint_name>"
)
!{command}

# Run Inference (Trained Model)

Final step to test your trained model:

1. Use the same `model_config.json`
2. Specify the unwrapped model checkpoint
3. Important: Use the unwrapped model name (`.ckpt`)

### Dataset Structure
- Dataset directory (recursively searched)
- `training_metadata.csv` containing:
  - Filename
  - Filepath
  - Additional training columns (customizable)
  
> 💡 **Note**: The custom dataset function can be modified to utilize different aspects of the training metadata

In [ ]:
%cd /workspace/stable-audio-tools

command = (
    "python run_gradio.py"
    " --model-config /workspace/model_config.json"
    " --ckpt-path <my_unwrapped_model_name>.ckpt"
    " --share"
)
!{command}